# 앙상블 학습 (Ensemble Learning)

**학습 목표:**
- "집단 지성"의 원리를 이용하는 앙상블 기법의 개념을 이해합니다. (여러 개의 약한 모델을 결합하여 강한 모델을 만드는 방법)
- 앙상블의 주요 두 가지 접근법인 **배깅(Bagging)**과 **부스팅(Boosting)**의 차이를 학습합니다.
- 배깅의 대표 주자인 **랜덤 포레스트(Random Forest)**를 구현하고 특성 중요도를 확인합니다.
- 부스팅의 대표 주자이자 캐글 등 경진대회에서 최고의 성능을 보이는 **XGBoost**를 구현하고, **조기 종료(Early Stopping)** 기능을 사용해봅니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### (1) 데이터 준비
유방암 데이터셋을 사용하여, 단일 결정 트리, 랜덤 포레스트, XGBoost의 성능을 비교합니다.

In [ ]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156)

### (2) 랜덤 포레스트 (Random Forest)
랜덤 포레스트는 **배깅(Bagging, Bootstrap Aggregating)**을 기반으로 합니다. 훈련 데이터에서 중복을 허용하여 여러 개의 서브셋(bootstrap sample)을 만들고, 각 서브셋으로 개별 의사결정나무를 학습시킵니다. 예측 시에는 각 트리의 예측 결과를 다수결 투표(분류) 또는 평균(회귀)하여 최종 결정을 내립니다. 이 과정에서 모델의 분산이 줄어들어 일반화 성능이 향상됩니다.

In [ ]:
# 비교를 위한 단일 결정 트리
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)
pred_dt = dt_clf.predict(X_test)
acc_dt = accuracy_score(y_test, pred_dt)
print(f"Decision Tree Accuracy: {acc_dt:.4f}")

# 랜덤 포레스트
rf_clf = RandomForestClassifier(n_estimators=100, random_state=156, n_jobs=-1)
rf_clf.fit(X_train, y_train)
pred_rf = rf_clf.predict(X_test)
acc_rf = accuracy_score(y_test, pred_rf)
print(f"Random Forest Accuracy: {acc_rf:.4f}")

### (3) XGBoost (eXtreme Gradient Boosting)
XGBoost는 **부스팅(Boosting)**을 기반으로 합니다. 첫 번째 모델이 예측하고 남긴 잔여 오차(residual)를 두 번째 모델이 학습하고, 다시 남은 오차를 세 번째 모델이 학습하는 방식으로, 순차적으로 약한 학습기들을 결합하여 모델을 점차 강화시킵니다. 특히 병렬 처리와 규제 기능을 통해 속도와 성능을 크게 향상시킨 알고리즘입니다.

**조기 종료(Early Stopping)**: 검증 세트(validation set)의 성능이 일정 횟수(early_stopping_rounds) 이상 개선되지 않으면, 불필요한 추가 학습을 멈춰 시간 낭비와 과적합을 방지합니다.

In [ ]:
# XGBoost는 검증 세트가 필요하므로 훈련 세트를 다시 나눔
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=156)

xgb_clf = xgb.XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3, 
                            use_label_encoder=False, eval_metric='logloss')

xgb_clf.fit(X_tr, y_tr, 
            early_stopping_rounds=100, # 100 라운드 동안 성능 향상 없으면 중단
            eval_set=[(X_val, y_val)], 
            verbose=False) # 학습 과정 출력 안 함

pred_xgb = xgb_clf.predict(X_test)
acc_xgb = accuracy_score(y_test, pred_xgb)
print(f"XGBoost Accuracy: {acc_xgb:.4f}")

### (4) 특성 중요도 시각화
앙상블 모델은 모델의 예측에 기여한 정도를 기준으로 특성 중요도를 제공하여, 어떤 특성이 중요한지 파악하는 데 도움을 줍니다.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
xgb.plot_importance(xgb_clf, ax=ax, max_num_features=10)
plt.title('XGBoost Feature Importances (Top 10)')
plt.show()